In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc') #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# dx = 250m; Np = 50M
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc') #***
parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc') #***
res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
##############################################
#DATA INITIALIZATION and LOADING FUNCTIONS

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
def initiate_array(out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g', 'A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
#INITIALIZE DATA FUNCTION
def InitiateArray_Job(data,out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel1['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g', 'A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(data1['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
# job_id=1;num_jobs=60
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# data=data1.isel(time=slice(start_job,end_job))
# parcel=parcel1.isel(time=slice(start_job,end_job))

In [ ]:
#################################################################################################################################################

In [ ]:
###############################################################
#FUNCTIONS

In [ ]:
def GetZYX(parcel):
    #Lagrangian Position Arrays
    ##############
    def grid_location(z,y,x):
        zf=data1['zf'].values*1000; which_zh=np.clip(np.searchsorted(zf,z)-1,0,None).astype(np.uint16)
        #which_zh=np.where(which_zh == -1, 0, which_zh) 
        
        yf=data1['yf'].values*1000; which_yh=np.clip(np.searchsorted(yf,y)-1,0,None).astype(np.uint16) 
        #which_yh=np.where(which_yh == -1, 0, which_yh) 
        
        xf=data1['xf'].values*1000; which_xh=np.clip(np.searchsorted(xf,x)-1,0,None).astype(np.uint16)
        #which_xh=np.where(which_xh == -1, 0, which_xh) 
        
        return which_zh,which_yh,which_xh
    
    print('Creating Lagrangian X,Y,Z Binary Arrays')
    x=parcel['x'].data;y=parcel['y'].data;z=parcel['z'].data
    Z,Y,X=grid_location(z,y,x)
    
    return Z,Y,X
# [Z,Y,X]=GetZYX(parcel)

In [ ]:
def call_variable(data,varname):
    if varname[0]=='w':
        var_data=data[varname].interp(zf=data['zh']).data
    else:
        var_data=data[varname].data
    return var_data

In [ ]:
# def MakeLagrangianArray(data,varnames,Z,Y,X): #SLOW FOR LOOP VERSION
#     print('Making Variables Lagrangian Array')
#     # Initialize dictionaries
#     var_data_dict = {varname: call_variable(data,varname) for varname in varnames}
#     VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

#     print('Running')
#     Nt = len(data['time'])
#     Np = len(parcel1['xh'])
#     for p in np.arange(Np):
#         if np.mod(p, 1e6) == 0: print(f"{p}/{len(parcel1['xh'])}")

#         # Get Indices
#         zs = Z[:, p]
#         ys = Y[:, p]
#         xs = X[:, p]
#         ts = np.arange(Nt)

#         # Loop over all variables and fill the respective VAR array
#         for varname, var_data in var_data_dict.items():
#             VAR[varname][:, p] = var_data[ts, zs, ys, xs]

#     # Return all the arrays in a list
#     # return [VAR[varname] for varname in varnames]
#     return VAR

def MakeLagrangianArray(data,varnames,Z,Y,X): #FAST ADVANCED INDEXING VERSION
    print('Making Variables Lagrangian Array')
    # Initialize dictionaries
    var_data_dict = {varname: call_variable(data,varname) for varname in varnames}
    VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

    print('Running')
    Nt = len(data['time'])
    # Get Indices
    z_idx = Z
    y_idx = Y
    x_idx = X
    ts = np.arange(Nt)[:, None]
    t_idx = np.broadcast_to(ts, Z.shape)

    for varname, var_data in var_data_dict.items():
        VAR[varname] = var_data[t_idx, z_idx, y_idx, x_idx]

    # return [VAR[varname] for varname in varnames]
    return VAR

In [ ]:
def SaveData(data,varnames,VAR,   var_type,job_id):
    # Saving Data
    ##############
    print('Saving Data\n')
    import h5py
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
    out_file=dir2+f'lagrangian_{var_type}_BUDGET_VARS_array_{res}_{t_res}_{Np_str}'
    out_file+=f'_{job_id}.h5'

    out_varnames=[varname.upper() for varname in varnames]
    InitiateArray_Job(data,out_file,out_varnames,t_chunk_size=1,p_chunk_size=1_000_000)
    with h5py.File(out_file, 'a') as f: 
        for var in VAR:
            f[var.upper()][:] = VAR[var]
# SaveData(data,varnames,VAR)            

In [ ]:
################################
#RUNNING

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if res == "1km" and Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=60
elif res == "1km" and Np_str=='50e6':
    num_jobs=400 #50M parcels
    num_slurm_jobs=60
elif res == "250m" and Np_str=='50e6':
    num_jobs=600 #50M parcels
    num_slurm_jobs=150
##############################

In [ ]:
################################
#RUN SETUP
###############################
varnames_W=['wb_hadv','wb_vadv','wb_hidiff','wb_vidiff',
          'wb_hturb','wb_vturb','wb_pgrad','wb_buoy']
varnames_QV=['qvb_hadv','qvb_vadv','qvb_hidiff','qvb_vidiff',
          'qvb_hturb','qvb_vturb','qvb_mp']
varnames_TH=['ptb_hadv','ptb_vadv','ptb_hidiff','ptb_vidiff',
          'ptb_hturb','ptb_vturb','ptb_mp','ptb_rad','ptb_div','ptb_diss']
varnames_WQVTH=varnames_W+varnames_QV+varnames_TH

def GetVarnames(var_type):
    varnames=globals()[f"varnames_{var_type}"]
    return varnames

In [ ]:
def Run(var_type):
    [start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=False) #if ISRUN is False, then will not run using slurm_job_array
    print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")
    
    job_id_list=np.arange(start_slurm_job,end_slurm_job)
    for job_id in job_id_list:
        if job_id % 1 == 0: print(f'current job_id = {job_id}/{num_jobs}')
        [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
    
        #SLICING DATA
        data=data1.isel(time=slice(start_job,end_job))
        parcel=parcel1.isel(time=slice(start_job,end_job))
    
        #GETTING POSITION VARIABLES
        [Z,Y,X]=GetZYX(parcel)

        #GET VARNAMES
        varnames=GetVarnames(var_type)
    
        #MAKING LAGRANGIAN ARRAYS (for threshold)
        VAR=MakeLagrangianArray(data,varnames,Z,Y,X)
    
        #SAVING
        SaveData(data,varnames,VAR, var_type,job_id)
    
        #check_memory(globals())
        del VAR

In [ ]:
#RUNNING
for var_type in ['W','QV','TH']:
# for var_type in ['WQVTH']: #̨MEMORY ERROR
    print(f"running for var_type: {var_type}")
    Run(var_type)

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
# recombine=True

In [ ]:
# #OLD
# def Recombine(num_jobs,var_type):
#     dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
#     dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
#     out_file=dir3+f'lagrangian_{var_type}_BUDGET_VARS_array_{res}_{t_res}_{Np_str}.h5'

#     print('initializing array')
#     vars=['u','w','z','Z','Y','X','W','QCQI','A_g','A_c']
#     InitiateArray_Job(out_file,vars,t_chunk_size=100,p_chunk_size=500_000)

#     print('recombining')
#     with h5py.File(out_file, 'r+') as f_out:
#         for job_id in np.arange(1,num_jobs+1):
#             if job_id % 5: print(f"job_id = {job_id}")
#             [a,b] = get_job_range(job_id,num_jobs)
    
#             in_file=dir2+f'lagrangian_{var_type}_BUDGET_VARS_array_{res}_{t_res}_{Np_str}_{job_id}.h5' 
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][a:b]=f_in[var][:]

In [ ]:
def Recombine_Dask(var_type):
    #DASK-ENABLED
    import glob
    import re
    def recombine_func(in_files,out_file):
        # matching_files = sorted(glob.glob(in_files))
        matching_files = sorted(
        glob.glob(in_files),
        key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
    )
        
        print('recombining')
        out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_0',combine='nested',phony_dims='sort')
        from dask.diagnostics import ProgressBar
        with ProgressBar():
            out.to_netcdf(out_file, engine='h5netcdf')
        
    if recombine==True:
        dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
        dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
        in_files = dir2 + f'lagrangian_{var_type}_BUDGET_VARS_array_{res}_{t_res}_{Np_str}_*.h5'
        out_file=dir3+f'lagrangian_{var_type}_BUDGET_VARS_array_{res}_{t_res}_{Np_str}.h5' 
        
        recombine_func(in_files,out_file)

In [ ]:
if recombine==True:
    for var_type in ['W','QV','TH']:
    # for var_type in ['WQVTH']:
        # Recombine(num_jobs=num_jobs,var_type=var_type)
        Recombine_Dask(var_type)

In [ ]:
##########################################################################################

In [ ]:
# #LOADING BACK IN
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# out_file=dir2+f'lagrangian_BUDGET_VARS_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(open_file, 'r') as f:
#     # Load the dataset by its name
#     VAR = f['VAR'][start_job:end_job]
#     # for key in f:
#         # globals()[key]=f[key][start_job:end_job]
# check_memory(globals())

In [ ]:
########################
# TESTING

In [ ]:
# #TESTING (IS VERSION3==VERSION2)
# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# # start_job,end_job=0,3

# #TESTING
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# open_file=dir2+f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(open_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"1"]=f[key][start_job:end_job]
#     # QV = f['QV'][start_job:end_job]
#     # TH = f['TH'][start_job:end_job]
#     # TH_E = f['TH_E'][start_job:end_job]
#     # BUOYANCY = f['BUOYANCY'][start_job:end_job]
#     # HMC = f['HMC'][start_job:end_job]
#     # VMF_c = f['VMF_c'][start_job:end_job]
# check_memory(globals())

# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
# out_file=dir2+f'lagrangian_W_BUDGET_VARS_array_{res}_{t_res}_{Np_str}'
# out_file+=f'_{job_id}.h5'
# with h5py.File(out_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"2"]=f[key][:]
# check_memory(globals())

# print(np.all(WB_BUOY1==WB_BUOY2))
# print(np.all(WB_HADV1==WB_HADV2))

In [ ]:
# #TESTING (IS VERSION3==VERSION2)
# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# # start_job,end_job=0,3

# #TESTING
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# open_file=dir2+f'QV_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(open_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"1"]=f[key][start_job:end_job]
#     # QV = f['QV'][start_job:end_job]
#     # TH = f['TH'][start_job:end_job]
#     # TH_E = f['TH_E'][start_job:end_job]
#     # BUOYANCY = f['BUOYANCY'][start_job:end_job]
#     # HMC = f['HMC'][start_job:end_job]
#     # VMF_c = f['VMF_c'][start_job:end_job]
# check_memory(globals())

# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
# out_file=dir2+f'lagrangian_QV_BUDGET_VARS_array_{res}_{t_res}_{Np_str}'
# out_file+=f'_{job_id}.h5'
# with h5py.File(out_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"2"]=f[key][:]
# check_memory(globals())

# print(np.all(QVB_VADV1==QVB_VADV2))
# print(np.all(QVB_HADV1==QVB_HADV2))

In [ ]:
# #TESTING (IS VERSION3==VERSION2)
# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# # start_job,end_job=0,3

# #TESTING
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# open_file=dir2+f'TH_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(open_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"1"]=f[key][start_job:end_job]
#     # QV = f['QV'][start_job:end_job]
#     # TH = f['TH'][start_job:end_job]
#     # TH_E = f['TH_E'][start_job:end_job]
#     # BUOYANCY = f['BUOYANCY'][start_job:end_job]
#     # HMC = f['HMC'][start_job:end_job]
#     # VMF_c = f['VMF_c'][start_job:end_job]
# check_memory(globals())

# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
# out_file=dir2+f'lagrangian_TH_BUDGET_VARS_array_{res}_{t_res}_{Np_str}'
# out_file+=f'_{job_id}.h5'
# with h5py.File(out_file, 'r') as f:
#     # Load the dataset by its name
#     for key in f:
#         globals()[key+"2"]=f[key][:]
# check_memory(globals())

# print(np.all(PTB_VADV1==PTB_VADV2))
# print(np.all(PTB_HADV1==PTB_HADV2))

In [ ]:
# #TESTING ARE RESULTS CORRECT
# job_id=250;num_jobs=300
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
# # print(start_job,end_job)
# # start_job,end_job=0,3

# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# with h5py.File(dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5', 'r') as f:
#     # Load the dataset by its name
#     Z = f['Z'][start_job:end_job]
#     Y = f['Y'][start_job:end_job]
#     X = f['X'][start_job:end_job]

# #TESTING
# # var_type='W'
# # var_type='QV'
# var_type='TH'

# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
# open_file=dir2+f'lagrangian_{var_type}_BUDGET_VARS_array_{res}_{t_res}_{Np_str}_{job_id}.h5'
# with h5py.File(open_file, 'r') as f:
#     key_list=list(f.keys())
#     for key in f:
#         print(key)
#         globals()[key]=f[key][:]
# data2=data1.isel(time=slice(start_job,end_job))

In [ ]:
# #TESTING ARE RESULTS CORRECT
# t,p=0,10000
# def test(t,p,VAR,var):
#     z=Z[t,p];y=Y[t,p];x=X[t,p]
#     if key_list[0][0]=='W':
#         out = data2[var].isel(time=t, yh=y, xh=x).interp(zf=data2['zh']).isel(zh=z).data
#     else:
#         out=data2[var].isel(time=t,zh=z,yh=y,xh=x).data
#     print(VAR[t,p],out)
# for key in key_list:
#     test(t,p,globals()[key],key.lower())